In [42]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [43]:
df = pd.read_csv("D:/Code/py_code/Artificial-Neural-Network/Single-Layer-Perceptron/data/test_data_cl.csv", header=None, names=list(range(13)))
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NaN,NaN
1,892,3,Kelly,Mr. James,male,34.5,0,0,330911,78.292,NaN,Q,NaN
2,893,3,Wilkes,Mrs. James (Ellen Needs),female,47,1,0,363272,7,NaN,S,NaN
3,894,2,Myles,Mr. Thomas Francis,male,62,0,0,240276,96.875,NaN,Q,NaN
4,895,3,Wirz,Mr. Albert,male,27,0,0,315154,86.625,NaN,S,NaN


# Data Preprocessing

### Benerin column

In [44]:
col_name = df.iloc[0, :11].values
col_name = np.insert(col_name, 3, "FirstName")
df = df.drop(12, axis=1)

df.columns = col_name
df = df.drop(0).reset_index(drop=True)
df.head()

,PassengerId,Pclass,Name,FirstName,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,Kelly,Mr. James,male,34.5,0,0,330911,78.292,NaN,Q
1,893,3,Wilkes,Mrs. James (Ellen Needs),female,47,1,0,363272,7,NaN,S
2,894,2,Myles,Mr. Thomas Francis,male,62,0,0,240276,96.875,NaN,Q
3,895,3,Wirz,Mr. Albert,male,27,0,0,315154,86.625,NaN,S
4,896,3,Hirvonen,Mrs. Alexander (Helga E Lindqvist),female,22,1,1,3101298,122.875,NaN,S


### Merge Name dan FirstName

In [45]:
df["Name"] = df["FirstName"] + " " + df["Name"]
df = df.drop("FirstName", axis=1)
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,Mr. James Kelly,male,34.5,0,0,330911,78.292,NaN,Q
1,893,3,Mrs. James (Ellen Needs) Wilkes,female,47,1,0,363272,7,NaN,S
2,894,2,Mr. Thomas Francis Myles,male,62,0,0,240276,96.875,NaN,Q
3,895,3,Mr. Albert Wirz,male,27,0,0,315154,86.625,NaN,S
4,896,3,Mrs. Alexander (Helga E Lindqvist) Hirvonen,female,22,1,1,3101298,122.875,NaN,S


### Perbaikan data yang tidak sesuai

**Perbaikan Data Sex**

terdapat data dengan value yang bukan 'male' atau 'female'

In [46]:
print(df['Sex'].unique().tolist())
true_sex_val = ['male', 'female']
count = 0
for val in df["Sex"].values:
    if val not in true_sex_val:
        df.loc[count, 'Sex'] = np.random.choice(true_sex_val)
    count += 1
print(df['Sex'].unique().tolist())

['male', 'female', '45', '18.5', '36', '20', '16', '42', '29', '30', '46', '19', '0.17', '22', '21', nan]
['male', 'female']


**Perbaikan Data Fare**

terdapat data dengan value yang bukan float

In [47]:
count = 0
for val in df['Fare'].values:
    try:
        float(val)
    except ValueError:
        df.loc[count, 'Fare'] = 0
    count += 1

**Perbaikan Data Parch**

terdapat data dengan value yang bukan int

In [48]:
count = 0
for val in df['Parch'].values:
    try:
        if int(val) > 1:
            df.loc[count, 'Parch'] = 0
    except ValueError:
        df.loc[count, 'Parch'] = 0
    count += 1

**Remove missing id**

terdapat data dengan baris yang kosong

In [49]:
df_missing = df[df['PassengerId'].isnull()]
df = df.drop(df_missing.index, axis=0).reset_index(drop=True)
df.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
326,1301,3,Miss. Treasteall Peacock,female,3,1,1,SOTON/O.Q. 3101315,13.775,NaN,S
327,1303,1,Mrs. William Edward (Lillian E Thorpe) Minahan,female,37,1,0,19928,90,C78,Q
328,1304,3,Miss. Jenny Lovisa Henriksson,female,28,0,0,347086,7.775,NaN,S
329,1306,1,Dona. Fermina Oliva y Ocana,female,39,0,0,PC 17758,108.9,C105,C
330,1307,3,Mr. Simon Sivertsen Saether,male,38.5,0,0,SOTON/O.Q. 3101262,7.25,NaN,S


### Change Data Type

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  331 non-null    object
 1   Pclass       331 non-null    object
 2   Name         331 non-null    object
 3   Sex          331 non-null    object
 4   Age          331 non-null    object
 5   SibSp        331 non-null    object
 6   Parch        331 non-null    object
 7   Ticket       331 non-null    object
 8   Fare         318 non-null    object
 9   Cabin        100 non-null    object
 10  Embarked     317 non-null    object
dtypes: object(11)
memory usage: 28.6+ KB


In [51]:
df = df.astype(
    {"PassengerId": "int64", "Pclass": "int64", "SibSp": "int64", "Parch": "int64"}
)
df = df.astype(
    {
        "Name": "string",
        "Sex": "string",
        "Ticket": "string",
        "Cabin": "string",
        "Embarked": "string",
    }
)
df = df.astype({"Age": "float64", "Fare": "float64"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  331 non-null    int64  
 1   Pclass       331 non-null    int64  
 2   Name         331 non-null    string 
 3   Sex          331 non-null    string 
 4   Age          331 non-null    float64
 5   SibSp        331 non-null    int64  
 6   Parch        331 non-null    int64  
 7   Ticket       331 non-null    string 
 8   Fare         318 non-null    float64
 9   Cabin        100 non-null    string 
 10  Embarked     317 non-null    string 
dtypes: float64(2), int64(4), string(5)
memory usage: 28.6 KB


### Menghilangkan Column yang tidak dipakai

In [52]:
del df['Name']
del df['Ticket']
del df['Cabin']

### Cek & Replace Missing Value

In [53]:
missing = pd.DataFrame({
    'total' : df.isnull().sum(),
    'percent' : df.isnull().sum()/df.shape[0] * 100})
    
missing

,total,percent
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
Age,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Fare,13,3.927492
Embarked,14,4.229607


In [54]:
df['Embarked'] = df['Embarked'].fillna(value=np.random.choice(df['Embarked'].unique().tolist()))
df['Fare'] = df['Fare'].fillna(value=0)
df.isna().sum()
clean_data = df.copy()

### Encode

In [55]:
lbenc = LabelEncoder()

for i in df.columns.values:
    if df[i].dtypes == 'string':
        df[i] = lbenc.fit_transform(df[i])

df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,78.292,1
1,893,3,0,47.0,1,0,7.000,2
2,894,2,1,62.0,0,0,96.875,1
3,895,3,1,27.0,0,0,86.625,2
4,896,3,0,22.0,1,1,122.875,2


### Split Feature and ID

In [56]:
train = df.copy()
x_train = train.iloc[:, 1:].values
train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,78.292,1
1,893,3,0,47.0,1,0,7.000,2
2,894,2,1,62.0,0,0,96.875,1
3,895,3,1,27.0,0,0,86.625,2
4,896,3,0,22.0,1,1,122.875,2


### Import Test Label Data

In [57]:
label = pd.read_csv("D:/Code/py_code/Artificial-Neural-Network/Single-Layer-Perceptron/data/test_data_GroundTruth_cl.csv")
label = label['Survived'].values
label = label.reshape(-1,1)
label

array([[0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
    

### Normalisasi

In [58]:
feature = x_train.copy()

passengerId = train.iloc[:, 0].values.reshape(-1,1)
col = train.columns.values.tolist()
col.append("Survived")

mnmx = MinMaxScaler()
feature = mnmx.fit_transform(feature)

data = np.concatenate((passengerId, feature), axis=1)
dataLabel = np.concatenate((data, label), axis=1)
normalize_data = pd.DataFrame(dataLabel, columns=col)

normalize_data = normalize_data.astype({'PassengerId': 'int64', 'Survived': 'int64'})
normalize_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892,1.0,1.0,0.453947,0.000,0.0,0.094148,0.5,0
1,893,1.0,0.0,0.618421,0.125,0.0,0.008418,1.0,1
2,894,0.5,1.0,0.815789,0.000,0.0,0.116495,0.5,0
3,895,1.0,1.0,0.355263,0.000,0.0,0.104169,1.0,0
4,896,1.0,0.0,0.289474,0.125,1.0,0.147760,1.0,1


### Export hasil normalisasi ke excel

In [59]:
with pd.ExcelWriter('Data Testing.xlsx') as writer:
    clean_data.to_excel(writer, sheet_name='Hasil Preprocessing')
    df.to_excel(writer, sheet_name='Hasil Encode')
    normalize_data.to_excel(writer, sheet_name='Normalisasi')